Demonstrating how to get DonkeyCar Tub files into a PyTorch/fastai DataBlock

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *
from fastai.data.transforms import ColReader, Normalize, RandomSplitter
import torch

In [ ]:
from donkeycar.parts.tub_v2 import Tub
import pandas as pd
from pathlib import Path

In [ ]:
from malpi.dk.train import preprocessFileList, get_data, get_learner, get_autoencoder, train_autoencoder

In [ ]:
def learn_resnet():
    learn2 = cnn_learner(dls, resnet18, loss_func=MSELossFlat(), metrics=[rmse], cbs=ActivationStats(with_hist=True))
    learn2.fine_tune(5)
    
    learn2.recorder.plot_loss()
    learn2.show_results(figsize=(20,10))

The below code is modified from: https://github.com/cmasenas/fastai_navigation_training/blob/master/fastai_train.ipynb.

TODO: Figure out how to have multiple output heads

In [ ]:
def test_one_transform(name, inputs, df_all, batch_tfms, item_tfms, epochs, lr):
    dls = get_data(inputs, df_all=df_all, batch_tfms=batch_tfms, item_tfms=item_tfms)
    callbacks = [CSVLogger(f"Transform_{name}.csv", append=True)]
    learn = get_learner(dls)
    #learn.no_logging() #Try this to block logging when doing many training test runs
    learn.fit_one_cycle(epochs, lr, cbs=callbacks)
    #learn.recorder.plot_loss()
    #learn.show_results(figsize=(20,10))

In [ ]:
# Train multipel times using a list of Transforms, one at a time.
# Compare mean/stdev of best validation loss (or rmse?) for each Transform
df_all = get_dataframe("track1_warehouse.txt")
transforms = [None]
transforms.extend( [*aug_transforms(do_flip=False, size=128)] )
for tfm in transforms:
    name = "None" if tfm is None else str(tfm.__class__.__name__)
    print( f"Transform: {name}" )
    for i in range(5):
        print( f"   Run {i+1}" )
        test_one_transform(name, "track1_warehouse.txt", df_all, None, 5, 3e-3)

In [ ]:
def visualize_learner( learn ):
    #dls=nav.dataloaders(df, bs=512)
    preds, tgt = learn.get_preds(dl=[dls.one_batch()])

    plt.title("Target vs Predicted Steering", fontsize=18, y=1.0)
    plt.xlabel("Target", fontsize=14, labelpad=15)
    plt.ylabel("Predicted", fontsize=14, labelpad=15)
    plt.plot(tgt.T[0], preds.T[0],'bo')
    plt.plot([-1,1],[-1,1],'r', linewidth = 4)
    plt.show()

    plt.title("Target vs Predicted Throttle", fontsize=18, y=1.02)
    plt.xlabel("Target", fontsize=14, labelpad=15)
    plt.ylabel("Predicted", fontsize=14, labelpad=15)
    plt.plot(tgt.T[1], preds.T[1],'bo')
    plt.plot([0,1],[0,1],'r', linewidth = 4)
    plt.show()

In [ ]:
learn.export()

In [ ]:
df_all = get_dataframe("track1_warehouse.txt")
dls = get_data("track1_warehouse.txt", df_all=df_all, batch_tfms=None)

In [ ]:
learn = get_learner(dls)
learn.fit_one_cycle(15, 3e-3)

In [ ]:
visualize_learner(learn)

In [ ]:
learn.export('models/track1_v2.pkl')

In [ ]:
def clear_pyplot_memory():
    plt.clf()
    plt.cla()
    plt.close()

df_all = get_dataframe("track1_warehouse.txt")

transforms=[None,
            RandomResizedCrop(128,p=1.0,min_scale=0.5,ratio=(0.9,1.1)),
            RandomErasing(sh=0.2, max_count=6,p=1.0),
            Brightness(max_lighting=0.4, p=1.0),
            Contrast(max_lighting=0.4, p=1.0),
            Saturation(max_lighting=0.4, p=1.0)]
#dls = get_data(None, df_all, item_tfms=item_tfms, batch_tfms=batch_tfms)

for tfm in transforms:
    name = "None" if tfm is None else str(tfm.__class__.__name__)
    if name == "RandomResizedCrop":
        item_tfms = tfm
        batch_tfms = None
    else:
        item_tfms = None
        batch_tfms = tfm
        
    dls = get_data("track1_warehouse.txt",
                   df_all=df_all,
                   item_tfms=item_tfms, batch_tfms=batch_tfms)

    dls.show_batch(unique=True, show=True)
    plt.savefig( f'Transform_{name}.png' )
#clear_pyplot_memory()

In [ ]:
learn, dls = train_autoencoder( "tracks_all.txt", 5, 3e-3, name="ae_test1", verbose=False )

In [ ]:
learn.recorder.plot_loss()
learn.show_results(figsize=(20,10))
#plt.savefig(name + '.png')

In [ ]:
idx = 0

In [ ]:
idx += 1
im1 = dls.one_batch()[0]
im1_out = learn.model.forward(im1)
show_image(im1[idx])
show_image(im1_out[idx])